In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

import keras
import tensorflow as tf
from keras import backend as K 
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.callbacks import TensorBoard
from tensorflow.python.keras.engine.base_layer import Layer

Using TensorFlow backend.


In [234]:
sess = tf.InteractiveSession()

/usr/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [235]:
input_img = Input(shape=(28,28,1))

x = Conv2D(16, (3,3), activation = 'relu', padding = 'same')(input_img)
print(x.shape, input_img.shape)
x = MaxPooling2D((2,2),padding='same')(x)
print(x.shape)
x = Conv2D(16, (3,3), activation = 'relu', padding = 'same')(x)
print(x.shape)
x = MaxPooling2D((2,2),padding='same')(x)
print(x.shape)
x = MaxPooling2D((2,2),padding='valid')(x)

print(x.shape)

(?, 28, 28, 16) (?, 28, 28, 1)
(?, 14, 14, 16)
(?, 14, 14, 16)
(?, 7, 7, 16)
(?, 3, 3, 16)


In [33]:
tmp = Input(tensor=tf.random_normal([28,28,1]))
print(tmp.shape, input_img.shape)

(28, 28, 1) (?, 28, 28, 1)


In [1]:
def ReLU(X):
    return tf.maximum(X, np.zeros_like(X))

def softmax(x):
    return (np.exp(x - np.max(x))) / e_x.sum() 

def tanh(X):
    return tf.tanh(X)

In [242]:
def customConv2D(X, filters, kernel_size, strides=1, padding='valid', activation='ReLU'):
    
    if isinstance(kernel_size, tuple)==True:
        ker_h, ker_w = kernel_size
    else:
        ker_h, ker_w = kernel_size, kernel_size
        
    _, inp_h, inp_w, inp_c = X.shape
    
    if padding == 'valid':
        pad = 0
    elif padding == 'same':
        pad = (ker_h - 1)//2
    else:
        raise ValueError('Padding '+padding+' is not defined')
    
    conv_h = (inp_h + 2*pad - ker_h)//strides + 1
    conv_w = (inp_w + 2*pad - ker_w)//strides + 1
    
    X_pad = tf.pad(X, [[0,0],[pad,pad],[pad,pad],[0,0]])
    
    convolve = []
    for x in range(conv_h):
        for y in range(conv_w):
            c = tf.slice(X_pad, [0, x*strides, y*strides, 0], [-1, ker_h, ker_w, -1])
            convolve.append(c)
    
    X_mat = tf.reshape(tf.stack(convolve), [-1, ker_w*ker_h])
    
    W = tf.random_normal([ker_h, ker_w, int(inp_c), filters])
    W_mat = tf.reshape(W, [ker_h*ker_w, filters])
    
    b = tf.zeros([filters])
    
    print(X_mat.shape)
    conv = tf.matmul(X_mat, W_mat) + b
    print(conv.shape)
    conv = tf.transpose(tf.reshape(conv,[conv_h,conv_w,-1,filters]), [2,0,1,3])
    print(conv.shape)
    
    if activation:
        activation = activation.lower()
        if activation == 'relu':
            conv = ReLU(conv)
        elif activation == 'tanh':
            conv = tanh(conv)
        else:
            raise ValueError('Unknown activation')
    
    return conv
    
x1 = customConv2D(input_img, 16, (3,3), padding = 'same', activation = 'relu')
print(x1.shape)

(?, 9)
(?, 16)
(?, 28, 28, 16)
(?, 28, 28, 16)


In [243]:
def customMaxPool2D(X, kernel_size, strides=2, padding='valid'):
    
    if isinstance(kernel_size, tuple)==True:
        ker_h, ker_w = kernel_size
    else:
        ker_h, ker_w = kernel_size, kernel_size
    
    _, inp_h, inp_w, inp_c = X.shape
    
    if padding == 'valid':
        pad = 0
    elif padding == 'same':
        pad = (ker_h - 1)//2
    else:
        raise ValueError('Padding '+padding+' is not defined')
    
    pool_h = (inp_h + 2*pad - ker_h)//strides + 1 
    pool_w = (inp_w + 2*pad - ker_w)//strides + 1
    
    X_pad = tf.pad(X, [[0,0],[pad,pad],[pad,pad],[0,0]])
    
    pooling = []
    for x in range(pool_h):
        for y in range(pool_w):
            p = tf.slice(X_pad, [0, x*strides, y*strides, 0], [-1, ker_h, ker_w, -1])
            pooling.append(p)
    
    X_mat = tf.reshape(tf.stack(pooling), [-1, ker_w*ker_h])
    
    pool = tf.reduce_max(tf.reshape(X_mat, [pool_h, pool_w, -1, ker_h*ker_w, int(inp_c)]), axis = 3)
    return tf.transpose(pool, [2,0,1,3])

x2 = customMaxPool2D(x1, (2,2), strides = 2, padding='valid')
print(x2.shape)

(?, 14, 14, 16)


In [244]:
def customUpSampling2D(X, kernel_size, data_format='channels_last', interpolation='nearest'):
    
    if isinstance(kernel_size, tuple)==True:
        ker_h, ker_w = kernel_size
    else:
        ker_h, ker_w = kernel_size, kernel_size
        
    if (data_format=='channels_last'):
        n, inp_h, inp_w, inp_c = X.shape
    elif (data_format=='channels_first'):
        n, inp_c, inp_h, inp_w = X.shape
    else:
        raise ValueError('Data format is not valid')
    
    out_h = ker_h * inp_h
    out_w = ker_w * inp_w
 
    if (data_format=='channels_last'):
        img_up = np.zeros((out_h, out_w, inp_c))
    elif (data_format=='channels_first'):
        img_up = np.zeros((inp_c, out_h, out_w))
    
    for out_chan in range(inp_c):
        for x in range(inp_h):
            for y in range(inp_w):
                for i in range(ker_h):
                    for j in range(ker_w):
                        img_up[x*ker_h+i][y*ker_w+j][out_chan]=matrix[x][y][out_chan]
    return img_up

customUpSampling2D(x2, (2,2))

ValueError: setting an array element with a sequence.

In [245]:
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import image_ops

In [246]:
from collections import Counter

In [249]:
def customUpSam(X, kernel_size, data_format='channels_last', interpolation='nearest'):
    
    if isinstance(kernel_size, tuple)==True:
        ker_h, ker_w = kernel_size
    else:
        ker_h, ker_w = kernel_size, kernel_size
        
    if (data_format=='channels_last'):
        n, inp_h, inp_w, inp_c = X.shape
        rows, cols = 1, 2
    elif (data_format=='channels_first'):
        n, inp_c, inp_h, inp_w = X.shape
        rows, cols = 2, 3
    else:
        raise ValueError('Data format is not valid')
    
    out_h = ker_h * inp_h
    out_w = ker_w * inp_w
    
    #print(X.shape)
    orig_shape = X.shape
    #print(orig_shape)
    
    if orig_shape[rows] is None:
        new_height = None
    else:
        new_height = orig_shape[rows] * ker_h

    if orig_shape[cols] is None:
        new_width = None
    else:
        new_width = orig_shape[cols] * ker_w
    
    if data_format == 'channels_first':
        output_shape = (None, None, new_height, new_width)
    else:
        output_shape = (None, new_height, new_width, None)
        
    if data_format == 'channels_first':
        X = tf.transpose(X, [0, 2, 3, 1])
    #print(X.shape)
    if interpolation == 'nearest':
        old_shape = X.shape[rows:cols + 1]
        X_new = Input(shape = (int(new_height), int(new_width), int(inp_c)))
        print(X_new.shape)
        new_shape = X_new.shape[rows:cols + 1]
        #print(type(X), X.shape, old_shape, type(X_new), X_new.shape, new_shape)
        
        #row_ratio, col_ratio = ker_h, ker_w
        row_ratio, col_ratio = np.array(new_shape)//np.array(old_shape)
        #print(row_ratio)
        
        # row wise interpolation 
        tmp = np.array(range(1, 1 + int(old_shape[0] * row_ratio))) // int(row_ratio)
        print(tmp)
        tmp = np.ceil(tmp)
        row_idx = (np.ceil(tmp) - 1).astype(int)
        #print(row_idx)
        
        # column wise interpolation
        tmp = np.array(range(1, 1 + int(old_shape[1] * col_ratio))) // int(col_ratio)
        #print(tmp)
        col_idx = (np.ceil(tmp) - 1).astype(int)

        final_matrix = X[:, row_idx][col_idx, :]
        print(X.shape)
        for i in row_idx:
            final_matrix = final_matrix[:, i, :, :]
            #print(i)
        #final_matrix = tf.slice(X, [0, 0, col_idx, 0], [-1, ker_h, ker_w, -1])
        print(final_matrix)
        #X = image_ops.resize_nearest_neighbor(X, new_shape)
    #elif interpolation == 'bilinear':
        #X = image_ops.resize_bilinear(X, new_shape)
    else:
        raise ValueError('interpolation should be one '
                     'of "nearest" or "bilinear".')
    
    if data_format == 'channels_first':
        X = tf.transpose(X, [0, 3, 1, 2])
        
    #X.set_shape(output_shape)
    return X

customUpSam(x2, (2,2))

(?, 28, 28, 16)
[ 0  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11 12
 12 13 13 14]
(?, 14, 14, 16)


ValueError: Index out of range using input dim 3; input has only 3 dims for 'strided_slice_3494' (op: 'StridedSlice') with input shapes: [?,14,16], [4], [4], [4] and with computed input tensors: input[3] = <1 1 1 1>.

In [236]:
input_img.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])

In [238]:
t = Input(shape=(28,28,1))
print(input_img.shape, t.shape)

(?, 28, 28, 1) (?, 28, 28, 1)
